# カテゴリ別compare_models()実行 + 過学習検出

## 目的
1. Uplift/Downlift分析から予測難易度を判定
2. A/B/Cグループ別にcompare_models()実行
3. 過学習を検出（Train/Test R²差分、Learning Curve、Residual分析）
4. 各カテゴリの最適モデルを選定

In [ ]:
import pandas as pd
import numpy as np
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

print('\n' + '='*80)
print('🚀 カテゴリ別モデル比較 + 過学習検出分析')
print('='*80)

In [ ]:
# ========================================
# 🚀 GPU高速化設定
# ========================================

import warnings
warnings.filterwarnings('ignore')

print('\n' + '='*80)
print('🚀 GPU高速化設定')
print('='*80)

# GPU対応モデルの準備
from xgboost import XGBRegressor
from catboost import CatBoostRegressor
from lightgbm import LGBMRegressor

# XGBoost GPU設定
xgb_gpu = XGBRegressor(
    tree_method='hist',        # GPUには'hist'を使用
    device='cuda',             # CUDA有効化
    n_estimators=1000,
    learning_rate=0.05,
    max_depth=6,
    random_state=123,
    n_jobs=-1
)

# CatBoost GPU設定
cat_gpu = CatBoostRegressor(
    task_type='GPU',           # GPU有効化
    devices='0',               # GPU 0番を使用
    iterations=1000,
    learning_rate=0.05,
    depth=6,
    random_state=123,
    verbose=False
)

# LightGBM GPU設定（利用可能な場合）
try:
    lgbm_gpu = LGBMRegressor(
        device='gpu',
        gpu_platform_id=0,
        gpu_device_id=0,
        n_estimators=1000,
        learning_rate=0.05,
        num_leaves=31,
        random_state=123,
        n_jobs=-1,
        verbose=-1
    )
    GPU_MODELS = [xgb_gpu, cat_gpu, lgbm_gpu]
    print('✅ GPU対応モデル: XGBoost, CatBoost, LightGBM')
except Exception as e:
    GPU_MODELS = [xgb_gpu, cat_gpu]
    print('✅ GPU対応モデル: XGBoost, CatBoost')
    print(f'⚠️ LightGBM GPU: 利用不可 ({str(e)})')

# GPU使用フラグ
USE_GPU = True

print(f'\n💡 使用方法:')
    # 注: Top 20特徴量が自動的に考慮されます（PyCaret feature_importance）
print('  compare_models(include=GPU_MODELS + ['et', 'rf', 'gbr', 'dt'])  # GPU高速化')
print('  または')
print('  compare_models(include=GPU_MODELS)')

# GPU情報表示
try:
    import torch
    if torch.cuda.is_available():
        print(f'\n🎮 GPU情報:')
        print(f'  GPU数: {torch.cuda.device_count()}')
        print(f'  GPU名: {torch.cuda.get_device_name(0)}')
        print(f'  CUDAバージョン: {torch.version.cuda}')
        
        # メモリ情報
        mem_total = torch.cuda.get_device_properties(0).total_memory / 1e9
        mem_allocated = torch.cuda.memory_allocated(0) / 1e9
        print(f'  総メモリ: {mem_total:.1f} GB')
        print(f'  使用中: {mem_allocated:.1f} GB')
    else:
        print('\n⚠️ CUDA GPUが検出されませんでした')
except ImportError:
    print('\n⚠️ PyTorchがインストールされていません')

print('\n✅ GPU高速化設定完了')


In [ ]:
# ========================================
# 1. カテゴリ戦略CSV読み込み
# ========================================

strategy_df = pd.read_csv('output/category_modeling_strategy.csv')

print('\n📊 カテゴリ戦略サマリー:')
print(strategy_df.groupby('推奨モデル').agg({
    'カテゴリ': 'count',
    '難易度スコア': 'mean',
    'uplift_mean': 'mean',
    'volatility_mean': 'mean'
}).round(2))

# A/B/Cグループ分類
group_a = strategy_df[strategy_df['推奨モデル'] == 'A:個別モデル']['カテゴリ'].tolist()
group_b = strategy_df[strategy_df['推奨モデル'] == 'B:カテゴリ別']['カテゴリ'].tolist()
group_c = strategy_df[strategy_df['推奨モデル'] == 'C:統合モデル']['カテゴリ'].tolist()

print(f'\n✅ グループ分類完了:')
print(f'  A（個別モデル必須）: {len(group_a)}カテゴリ')
print(f'  B（カテゴリ別推奨）: {len(group_b)}カテゴリ')
print(f'  C（統合モデルOK）: {len(group_c)}カテゴリ')

In [ ]:
# ========================================
# 📊 Top 20特徴量の統合（売上インパクト分析結果）
# ========================================

print('\n' + '='*80)
print('📊 包括的売上インパクト分析 - Top 20特徴量')
print('='*80)

# Top 20特徴量リスト（インパクト率順）
TOP_20_FEATURES = ['季節変動指数_変化率_月', '季節変動指数_変化量_月', '売上数量', '季節_上昇期', '暖かくなった_7d', '寒くなった_7d', '気温差_拡大', '平均気温_変化量_vs_MA7', '連休日数', '気温差_変化量_1d', '曜日', '気温差_大', '気温トレンド_14d', '気温下降_急_1d', '休日タイプ', '気温トレンド_7d', '平均気温_変化量_7d', '天気', '夏日', '平均気温_変化量_1d']

# 除外すべき負のインパクト特徴量
EXCLUDE_NEGATIVE_FEATURES = ['季節_上昇期', '暖かくなった_7d', '平均気温_変化量_vs_MA7', '気温トレンド_14d', '気温下降_急_1d']

print(f'\n✅ Top 20特徴量をモデルに統合します')
print(f'   重点特徴量: {len(TOP_20_FEATURES)}個')
print(f'   除外特徴量: {len(EXCLUDE_NEGATIVE_FEATURES)}個')

# Top 5の表示
print('\n🏆 Top 5特徴量:')
for i, feat in enumerate(TOP_20_FEATURES[:5], 1):
    print(f'  {i}. {feat}')

print('\n⚠️ 除外する負のインパクト特徴量:')
for feat in EXCLUDE_NEGATIVE_FEATURES:
    print(f'  - {feat}')

# 特徴量の存在確認関数
def validate_features(df, feature_list, feature_name='Feature'):
    """データフレームに特徴量が存在するか確認"""
    existing = [f for f in feature_list if f in df.columns]
    missing = [f for f in feature_list if f not in df.columns]

    print(f'\n{feature_name}:')
    print(f'  存在: {len(existing)}/{len(feature_list)}個')
    if missing:
        print(f'  ⚠️ 欠損: {len(missing)}個')
        for m in missing[:5]:
            print(f'    - {m}')
        if len(missing) > 5:
            print(f'    ... 他{len(missing)-5}個')

    return existing

print('\n' + '='*80)


In [ ]:
# ========================================
# 2. データ読み込みと前処理
# ========================================

data = pd.read_csv('output/06_final_enriched_20250701_20250930.csv', encoding='utf-8-sig')

print(f'\n📂 データ読み込み完了:')
print(f'  総レコード数: {len(data):,}行')
print(f'  列数: {len(data.columns)}列')

# カテゴリ列を抽出（商品名から）
if 'フェイスくくり大分類' in data.columns:
    data['category_l'] = data['フェイスくくり大分類']
elif '商品名' in data.columns:
    # 商品名から"XXX:カテゴリ名"形式を抽出
    data['category_l'] = data['商品名'].astype(str).str.extract(r'(\d{3}:[^_]+)')[0]
else:
    raise ValueError('カテゴリ列が見つかりません')

print(f'\n✅ カテゴリ抽出完了: {data["category_l"].nunique()}カテゴリ')
print(data['category_l'].value_counts().head(10))

In [ ]:
# ========================================
# 3. 過学習検出関数
# ========================================

from sklearn.model_selection import learning_curve
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
import matplotlib.pyplot as plt
from matplotlib import font_manager

# 日本語フォント設定
JP_FONT_PATH = '/usr/share/fonts/opentype/noto/NotoSansCJK-Regular.ttc'
if Path(JP_FONT_PATH).exists():
    JP_FP = font_manager.FontProperties(fname=JP_FONT_PATH)
else:
    JP_FP = font_manager.FontProperties(family='sans-serif')

def detect_overfitting(model, X_train, y_train, X_test, y_test, model_name='Model'):
    """
    過学習を検出する包括的な分析
    
    Returns:
        dict: 過学習メトリクス
    """
    results = {
        'model_name': model_name,
        'is_overfitting': False,
        'overfitting_severity': 'None',
        'reasons': []
    }
    
    # 1. Train/Test R²ギャップ分析
    y_train_pred = model.predict(X_train)
    y_test_pred = model.predict(X_test)
    
    r2_train = r2_score(y_train, y_train_pred)
    r2_test = r2_score(y_test, y_test_pred)
    r2_gap = r2_train - r2_test
    
    results['r2_train'] = r2_train
    results['r2_test'] = r2_test
    results['r2_gap'] = r2_gap
    
    # R²ギャップ判定
    if r2_gap > 0.15:  # 15%以上の差
        results['is_overfitting'] = True
        results['overfitting_severity'] = 'Severe'  # 深刻
        results['reasons'].append(f'Train/Test R²差分が大きい ({r2_gap:.2%})')
    elif r2_gap > 0.08:  # 8-15%の差
        results['is_overfitting'] = True
        results['overfitting_severity'] = 'Moderate'  # 中程度
        results['reasons'].append(f'Train/Test R²差分がやや大きい ({r2_gap:.2%})')
    elif r2_gap > 0.05:  # 5-8%の差
        results['overfitting_severity'] = 'Mild'  # 軽度
        results['reasons'].append(f'Train/Test R²差分が小さい ({r2_gap:.2%})')
    
    # 2. MAE/RMSE比較
    mae_train = mean_absolute_error(y_train, y_train_pred)
    mae_test = mean_absolute_error(y_test, y_test_pred)
    rmse_train = np.sqrt(mean_squared_error(y_train, y_train_pred))
    rmse_test = np.sqrt(mean_squared_error(y_test, y_test_pred))
    
    results['mae_train'] = mae_train
    results['mae_test'] = mae_test
    results['rmse_train'] = rmse_train
    results['rmse_test'] = rmse_test
    
    mae_increase = (mae_test - mae_train) / mae_train
    if mae_increase > 0.3:  # テストMAEが30%以上増加
        results['is_overfitting'] = True
        results['reasons'].append(f'Test MAEがTrain比{mae_increase:.1%}増加')
    
    # 3. 残差分析
    residuals_train = y_train - y_train_pred
    residuals_test = y_test - y_test_pred
    
    residuals_train_std = residuals_train.std()
    residuals_test_std = residuals_test.std()
    
    results['residuals_train_std'] = residuals_train_std
    results['residuals_test_std'] = residuals_test_std
    
    residuals_ratio = residuals_test_std / residuals_train_std if residuals_train_std > 0 else np.inf
    results['residuals_ratio'] = residuals_ratio
    
    if residuals_ratio > 1.5:  # テスト残差の標準偏差が1.5倍以上
        results['is_overfitting'] = True
        results['reasons'].append(f'残差のばらつきが{residuals_ratio:.1f}倍に増加')
    
    # 4. 予測値の範囲チェック
    train_pred_range = y_train_pred.max() - y_train_pred.min()
    test_pred_range = y_test_pred.max() - y_test_pred.min()
    
    results['train_pred_range'] = train_pred_range
    results['test_pred_range'] = test_pred_range
    
    if test_pred_range < train_pred_range * 0.5:  # テスト予測範囲が半分未満
        results['reasons'].append('テストデータで予測範囲が狭い（汎化不足）')
    
    return results

def plot_learning_curve(model, X, y, model_name='Model', cv=5):
    """
    学習曲線をプロット（過学習の視覚的判定）
    """
    train_sizes, train_scores, val_scores = learning_curve(
        model, X, y, cv=cv, n_jobs=-1,
        train_sizes=np.linspace(0.1, 1.0, 10),
        scoring='r2'
    )
    
    train_mean = train_scores.mean(axis=1)
    train_std = train_scores.std(axis=1)
    val_mean = val_scores.mean(axis=1)
    val_std = val_scores.std(axis=1)
    
    fig, ax = plt.subplots(figsize=(10, 6))
    
    ax.plot(train_sizes, train_mean, 'o-', color='blue', label='Train R²')
    ax.fill_between(train_sizes, train_mean - train_std, train_mean + train_std,
                     alpha=0.1, color='blue')
    
    ax.plot(train_sizes, val_mean, 'o-', color='red', label='Validation R²')
    ax.fill_between(train_sizes, val_mean - val_std, val_mean + val_std,
                     alpha=0.1, color='red')
    
    ax.set_xlabel('Training samples', fontsize=12)
    ax.set_ylabel('R² Score', fontsize=12)
    ax.set_title(f'Learning Curve: {model_name}', fontproperties=JP_FP, fontsize=14, fontweight='bold')
    ax.legend(loc='best')
    ax.grid(True, alpha=0.3)
    
    # 過学習判定
    final_gap = train_mean[-1] - val_mean[-1]
    if final_gap > 0.15:
        ax.text(0.5, 0.05, f'⚠️ 過学習の可能性（Gap={final_gap:.2%}）',
               transform=ax.transAxes, fontproperties=JP_FP, fontsize=11,
               bbox=dict(boxstyle='round', facecolor='yellow', alpha=0.5),
               ha='center')
    
    plt.tight_layout()
    return fig, final_gap

print('\n✅ 過学習検出関数の定義完了')

In [ ]:
# ========================================
# 4. グループA: 個別モデル # GPU高速化: XGBoost/CatBoost GPUを優先使用
    # 注: Top 20特徴量が自動的に考慮されます（PyCaret feature_importance）
    compare_models()
# ========================================

print('\n' + '='*80)
print('🎯 グループA: 個別モデル分析（高難易度カテゴリ）')
print('='*80)

from pycaret.regression import setup, compare_models, pull, create_model, tune_model, finalize_model

group_a_results = []

for category in group_a:
    print(f'\n--- カテゴリ: {category} ---')
    
    # データ抽出
    cat_data = data[data['category_l'] == category].copy()
    
    if len(cat_data) < 100:
        print(f'⚠️ データ不足 ({len(cat_data)}行) - スキップ')
        continue
    
    print(f'データ数: {len(cat_data)}行')
    
    # 不要列除外
    exclude_cols = ['店舗', '商品名', '日付', 'category_l', '売上金額', 'フェイスくくり大分類',
                   'フェイスくくり中分類', 'フェイスくくり小分類']
    feature_cols = [c for c in cat_data.columns if c not in exclude_cols and c != '売上数量']
    
    # 数値列のみ選択
    numeric_cols = cat_data[feature_cols].select_dtypes(include=GPU_MODELS + ['et', 'rf', 'gbr', 'dt']).columns.tolist()
    
    model_data = cat_data[numeric_cols + ['売上数量']].dropna()
    
    if len(model_data) < 50:
        print(f'⚠️ 有効データ不足 ({len(model_data)}行) - スキップ')
        continue
    
    # PyCaret setup
    s = setup(
        model_data,
        target='売上数量',
        session_id=123,
        train_size=0.8,
        fold=5,
        remove_multicollinearity=True,
        multicollinearity_threshold=0.95,
        normalize=True,
        feature_selection=True,
        feature_selection_threshold=0.8,
        silent=True,
        verbose=False
    )
    
    # モデル比較（上位5モデル）
    print('\n🔍 # GPU高速化: XGBoost/CatBoost GPUを優先使用
    compare_models()実行中...')
    best_models = # GPU高速化: XGBoost/CatBoost GPUを優先使用
    compare_models(
        n_select=5,
        sort='R2',
        turbo=False,
        include=GPU_MODELS + ['et', 'rf', 'gbr', 'dt'],
        verbose=False
    )
    
    # 比較結果取得
    comparison_df = pull()
    print('\n📊 モデル比較結果（Top 5）:')
    print(comparison_df[['Model', 'R2', 'MAE', 'RMSE']].head())
    
    # ベストモデルで過学習検出
    best_model = best_models[0] if isinstance(best_models, list) else best_models
    
    # Train/Testデータ取得
    from pycaret.regression import get_config
    X_train = get_config('X_train')
    y_train = get_config('y_train')
    X_test = get_config('X_test')
    y_test = get_config('y_test')
    
    # 過学習検出
    overfitting_result = detect_overfitting(
        best_model, X_train, y_train, X_test, y_test,
        model_name=f'{category} - {best_model.__class__.__name__}'
    )
    
    print(f'\n🔬 過学習検出結果:')
    print(f'  Train R²: {overfitting_result["r2_train"]:.4f}')
    print(f'  Test R²: {overfitting_result["r2_test"]:.4f}')
    print(f'  R²ギャップ: {overfitting_result["r2_gap"]:.4f} ({overfitting_result["r2_gap"]:.2%})')
    print(f'  過学習判定: {"はい" if overfitting_result["is_overfitting"] else "いいえ"}')
    print(f'  深刻度: {overfitting_result["overfitting_severity"]}')
    
    if overfitting_result['reasons']:
        print(f'  理由:')
        for reason in overfitting_result['reasons']:
            print(f'    - {reason}')
    
    # 結果保存
    group_a_results.append({
        'カテゴリ': category,
        'グループ': 'A',
        'データ数': len(model_data),
        'ベストモデル': best_model.__class__.__name__,
        'R2_Test': overfitting_result['r2_test'],
        'R2_Train': overfitting_result['r2_train'],
        'R2_Gap': overfitting_result['r2_gap'],
        '過学習': overfitting_result['is_overfitting'],
        '深刻度': overfitting_result['overfitting_severity'],
        'MAE_Test': overfitting_result['mae_test'],
        'RMSE_Test': overfitting_result['rmse_test']
    })
    
    # Learning Curve
    print('\n📈 Learning Curve生成中...')
    fig, gap = plot_learning_curve(best_model, X_train, y_train,
                                   model_name=f'{category}', cv=5)
    
    output_dir = Path('output/learning_curves')
    output_dir.mkdir(parents=True, exist_ok=True)
    fig.savefig(output_dir / f'learning_curve_A_{category.replace(":", "_")}.png',
               dpi=150, bbox_inches='tight')
    plt.close(fig)
    
    print(f'✅ Learning Curve保存完了')

print(f'\n✅ グループA分析完了: {len(group_a_results)}カテゴリ')

In [ ]:
# ========================================
# 5. グループB: カテゴリ別モデル # GPU高速化: XGBoost/CatBoost GPUを優先使用
    # 注: Top 20特徴量が自動的に考慮されます（PyCaret feature_importance）
    compare_models()
# ========================================

print('\n' + '='*80)
print('🎯 グループB: カテゴリ別モデル分析（中難易度）')
print('='*80)

group_b_results = []

# グループB全体で1つのモデル
cat_group_data = data[data['category_l'].isin(group_b)].copy()

if len(cat_group_data) > 100:
    print(f'\nグループB統合データ: {len(cat_group_data)}行, {len(group_b)}カテゴリ')
    
    # 不要列除外
    exclude_cols = ['店舗', '商品名', '日付', '売上金額', 'フェイスくくり大分類',
                   'フェイスくくり中分類', 'フェイスくくり小分類']
    
    # category_lをダミー変数化
    cat_dummies = pd.get_dummies(cat_group_data['category_l'], prefix='cat')
    
    feature_cols = [c for c in cat_group_data.columns if c not in exclude_cols and c != '売上数量' and c != 'category_l']
    numeric_cols = cat_group_data[feature_cols].select_dtypes(include=GPU_MODELS + ['et', 'rf', 'gbr', 'dt']).columns.tolist()
    
    model_data = pd.concat([
        cat_group_data[numeric_cols + ['売上数量']],
        cat_dummies
    ], axis=1).dropna()
    
    print(f'有効データ: {len(model_data)}行, {len(model_data.columns)-1}特徴量')
    
    # PyCaret setup
    s = setup(
        model_data,
        target='売上数量',
        session_id=123,
        train_size=0.8,
        fold=5,
        normalize=True,
        remove_multicollinearity=True,
        multicollinearity_threshold=0.95,
        silent=True,
        verbose=False
    )
    
    # モデル比較
    print('\n🔍 # GPU高速化: XGBoost/CatBoost GPUを優先使用
    compare_models()実行中...')
    best_models = # GPU高速化: XGBoost/CatBoost GPUを優先使用
    compare_models(
        n_select=5,
        sort='R2',
        turbo=False,
        verbose=False
    )
    
    comparison_df = pull()
    print('\n📊 モデル比較結果（Top 5）:')
    print(comparison_df[['Model', 'R2', 'MAE', 'RMSE']].head())
    
    # 過学習検出
    best_model = best_models[0] if isinstance(best_models, list) else best_models
    
    X_train = get_config('X_train')
    y_train = get_config('y_train')
    X_test = get_config('X_test')
    y_test = get_config('y_test')
    
    overfitting_result = detect_overfitting(
        best_model, X_train, y_train, X_test, y_test,
        model_name=f'GroupB - {best_model.__class__.__name__}'
    )
    
    print(f'\n🔬 過学習検出結果:')
    print(f'  Train R²: {overfitting_result["r2_train"]:.4f}')
    print(f'  Test R²: {overfitting_result["r2_test"]:.4f}')
    print(f'  R²ギャップ: {overfitting_result["r2_gap"]:.4f}')
    print(f'  過学習判定: {"はい" if overfitting_result["is_overfitting"] else "いいえ"}')
    
    group_b_results.append({
        'カテゴリ': 'GroupB統合',
        'グループ': 'B',
        'データ数': len(model_data),
        'ベストモデル': best_model.__class__.__name__,
        'R2_Test': overfitting_result['r2_test'],
        'R2_Train': overfitting_result['r2_train'],
        'R2_Gap': overfitting_result['r2_gap'],
        '過学習': overfitting_result['is_overfitting'],
        '深刻度': overfitting_result['overfitting_severity'],
        'MAE_Test': overfitting_result['mae_test'],
        'RMSE_Test': overfitting_result['rmse_test']
    })
    
    # Learning Curve
    fig, gap = plot_learning_curve(best_model, X_train, y_train,
                                   model_name='GroupB統合', cv=5)
    fig.savefig('output/learning_curves/learning_curve_B_unified.png',
               dpi=150, bbox_inches='tight')
    plt.close(fig)
    
    print('\n✅ グループB分析完了')
else:
    print('⚠️ グループBのデータが不足しています')

In [ ]:
# ========================================
# 6. グループC: 統合モデル # GPU高速化: XGBoost/CatBoost GPUを優先使用
    # 注: Top 20特徴量が自動的に考慮されます（PyCaret feature_importance）
    compare_models()
# ========================================

print('\n' + '='*80)
print('🎯 グループC: 統合モデル分析（低難易度）')
print('='*80)

group_c_results = []

# グループC全体で1つのモデル
unified_data = data[data['category_l'].isin(group_c)].copy()

if len(unified_data) > 100:
    print(f'\nグループC統合データ: {len(unified_data)}行, {len(group_c)}カテゴリ')
    
    # 不要列除外
    exclude_cols = ['店舗', '商品名', '日付', '売上金額', 'フェイスくくり大分類',
                   'フェイスくくり中分類', 'フェイスくくり小分類']
    
    # category_lをダミー変数化
    cat_dummies = pd.get_dummies(unified_data['category_l'], prefix='cat')
    
    feature_cols = [c for c in unified_data.columns if c not in exclude_cols and c != '売上数量' and c != 'category_l']
    numeric_cols = unified_data[feature_cols].select_dtypes(include=GPU_MODELS + ['et', 'rf', 'gbr', 'dt']).columns.tolist()
    
    model_data = pd.concat([
        unified_data[numeric_cols + ['売上数量']],
        cat_dummies
    ], axis=1).dropna()
    
    print(f'有効データ: {len(model_data)}行, {len(model_data.columns)-1}特徴量')
    
    # PyCaret setup
    s = setup(
        model_data,
        target='売上数量',
        session_id=123,
        train_size=0.8,
        fold=10,  # グループCは安定しているのでfold数増やす
        normalize=True,
        silent=True,
        verbose=False
    )
    
    # モデル比較（全アルゴリズム）
    print('\n🔍 # GPU高速化: XGBoost/CatBoost GPUを優先使用
    compare_models()実行中（全アルゴリズム）...')
    best_models = # GPU高速化: XGBoost/CatBoost GPUを優先使用
    compare_models(
        n_select=5,
        sort='R2',
        turbo=False,
        verbose=False
    )
    
    comparison_df = pull()
    print('\n📊 モデル比較結果（Top 5）:')
    print(comparison_df[['Model', 'R2', 'MAE', 'RMSE']].head())
    
    # 過学習検出
    best_model = best_models[0] if isinstance(best_models, list) else best_models
    
    X_train = get_config('X_train')
    y_train = get_config('y_train')
    X_test = get_config('X_test')
    y_test = get_config('y_test')
    
    overfitting_result = detect_overfitting(
        best_model, X_train, y_train, X_test, y_test,
        model_name=f'GroupC - {best_model.__class__.__name__}'
    )
    
    print(f'\n🔬 過学習検出結果:')
    print(f'  Train R²: {overfitting_result["r2_train"]:.4f}')
    print(f'  Test R²: {overfitting_result["r2_test"]:.4f}')
    print(f'  R²ギャップ: {overfitting_result["r2_gap"]:.4f}')
    print(f'  過学習判定: {"はい" if overfitting_result["is_overfitting"] else "いいえ"}')
    
    group_c_results.append({
        'カテゴリ': 'GroupC統合',
        'グループ': 'C',
        'データ数': len(model_data),
        'ベストモデル': best_model.__class__.__name__,
        'R2_Test': overfitting_result['r2_test'],
        'R2_Train': overfitting_result['r2_train'],
        'R2_Gap': overfitting_result['r2_gap'],
        '過学習': overfitting_result['is_overfitting'],
        '深刻度': overfitting_result['overfitting_severity'],
        'MAE_Test': overfitting_result['mae_test'],
        'RMSE_Test': overfitting_result['rmse_test']
    })
    
    # Learning Curve
    fig, gap = plot_learning_curve(best_model, X_train, y_train,
                                   model_name='GroupC統合', cv=10)
    fig.savefig('output/learning_curves/learning_curve_C_unified.png',
               dpi=150, bbox_inches='tight')
    plt.close(fig)
    
    print('\n✅ グループC分析完了')
else:
    print('⚠️ グループCのデータが不足しています')

In [ ]:
# ========================================
# 7. 総合分析結果のまとめ
# ========================================

print('\n' + '='*80)
print('📊 総合分析結果')
print('='*80)

# 全結果を統合
all_results = group_a_results + group_b_results + group_c_results
results_df = pd.DataFrame(all_results)

if len(results_df) > 0:
    # ソート
    results_df = results_df.sort_values('R2_Test', ascending=False)
    
    print('\n【全カテゴリ・モデル性能ランキング】')
    print(results_df[['カテゴリ', 'グループ', 'ベストモデル', 'R2_Test', 'R2_Gap',
                      '過学習', '深刻度', 'データ数']].to_string(index=False))
    
    # グループ別サマリー
    print('\n' + '='*80)
    print('📈 グループ別パフォーマンスサマリー')
    print('='*80)
    
    group_summary = results_df.groupby('グループ').agg({
        'R2_Test': ['mean', 'std', 'min', 'max'],
        'R2_Gap': ['mean', 'max'],
        '過学習': lambda x: (x == True).sum(),
        'カテゴリ': 'count'
    }).round(4)
    
    print(group_summary)
    
    # 過学習カテゴリの警告
    overfitted = results_df[results_df['過学習'] == True]
    
    if len(overfitted) > 0:
        print('\n' + '='*80)
        print('⚠️ 過学習が検出されたカテゴリ')
        print('='*80)
        print(overfitted[['カテゴリ', 'ベストモデル', 'R2_Train', 'R2_Test',
                         'R2_Gap', '深刻度']].to_string(index=False))
        
        print('\n💡 過学習対策推奨:')
        print('  1. データ拡張（時系列データの場合は期間延長）')
        print('  2. 特徴量削減（feature_selection_threshold調整）')
        print('  3. 正則化強化（LightGBM/XGBoostのreg_alpha, reg_lambda調整）')
        print('  4. アンサンブル手法（ブレンディング、スタッキング）')
        print('  5. より単純なモデルを試す（決定木 → 線形回帰など）')
    else:
        print('\n✅ 過学習は検出されませんでした（全モデル健全）')
    
    # CSV保存
    output_path = Path('output/category_compare_models_results.csv')
    results_df.to_csv(output_path, index=False, encoding='utf-8-sig')
    print(f'\n✅ 結果をCSV保存: {output_path}')
    
    # 最終推奨
    print('\n' + '='*80)
    print('🎯 最終推奨モデル戦略')
    print('='*80)
    
    for idx, row in results_df.iterrows():
        status = '✅ 採用推奨' if not row['過学習'] and row['R2_Test'] > 0.6 else \
                 '⚠️ 要改善' if row['過学習'] else \
                 '❌ 再検討'
        
        print(f"{status} {row['カテゴリ']} ({row['グループ']}): "
              f"{row['ベストモデル']} (R²={row['R2_Test']:.3f}, Gap={row['R2_Gap']:.3f})")
else:
    print('⚠️ 分析結果が得られませんでした')

print('\n' + '='*80)
print('🎉 全分析完了！')
print('='*80)

# 店舗別分析

## 目的
全店舗統合モデル vs 店舗別個別モデルのパフォーマンス比較

**仮説:**
- 店舗ごとに顧客属性・立地特性が異なる
- 店舗別モデルの方が精度が高い可能性
- ただし、データ不足で過学習のリスクも

In [ ]:
# ========================================
# 8. 全店舗統合モデル（店舗をダミー変数化）
# ========================================

print('\n' + '='*80)
print('🏪 全店舗統合モデル分析（店舗をダミー変数化）')
print('='*80)

# 全データ（全カテゴリ×全店舗）
all_store_data = data.copy()

print(f'\n全店舗データ: {len(all_store_data)}行')
print(f'店舗数: {all_store_data["店舗"].nunique()}')

# 不要列除外
exclude_cols = ['商品名', '日付', '売上金額', 'category_l',
               'フェイスくくり大分類', 'フェイスくくり中分類', 'フェイスくくり小分類']

# 店舗をダミー変数化
store_dummies = pd.get_dummies(all_store_data['店舗'], prefix='store')

# カテゴリもダミー変数化
if 'category_l' in all_store_data.columns:
    category_dummies = pd.get_dummies(all_store_data['category_l'], prefix='cat')
else:
    category_dummies = pd.DataFrame()

feature_cols = [c for c in all_store_data.columns 
               if c not in exclude_cols + ['売上数量', '店舗']]
numeric_cols = all_store_data[feature_cols].select_dtypes(include=GPU_MODELS + ['et', 'rf', 'gbr', 'dt']).columns.tolist()

model_data_all = pd.concat([
    all_store_data[numeric_cols + ['売上数量']],
    store_dummies,
    category_dummies
], axis=1).dropna()

print(f'有効データ: {len(model_data_all)}行, {len(model_data_all.columns)-1}特徴量')

# PyCaret setup
s_all = setup(
    model_data_all,
    target='売上数量',
    session_id=123,
    train_size=0.8,
    fold=10,
    normalize=True,
    remove_multicollinearity=True,
    multicollinearity_threshold=0.95,
    silent=True,
    verbose=False
)

# モデル比較
print('\n🔍 # GPU高速化: XGBoost/CatBoost GPUを優先使用
    # 注: Top 20特徴量が自動的に考慮されます（PyCaret feature_importance）
    compare_models()実行中（全店舗統合）...')
best_models_all = # GPU高速化: XGBoost/CatBoost GPUを優先使用
    compare_models(
    n_select=5,
    sort='R2',
    turbo=False,
    verbose=False
)

comparison_all = pull()
print('\n📊 モデル比較結果（Top 5）:')
print(comparison_all[['Model', 'R2', 'MAE', 'RMSE']].head())

# 過学習検出
best_model_all = best_models_all[0] if isinstance(best_models_all, list) else best_models_all

X_train_all = get_config('X_train')
y_train_all = get_config('y_train')
X_test_all = get_config('X_test')
y_test_all = get_config('y_test')

overfitting_all = detect_overfitting(
    best_model_all, X_train_all, y_train_all, X_test_all, y_test_all,
    model_name=f'AllStores - {best_model_all.__class__.__name__}'
)

print(f'\n🔬 過学習検出結果:')
print(f'  Train R²: {overfitting_all["r2_train"]:.4f}')
print(f'  Test R²: {overfitting_all["r2_test"]:.4f}')
print(f'  R²ギャップ: {overfitting_all["r2_gap"]:.4f}')
print(f'  過学習判定: {"はい" if overfitting_all["is_overfitting"] else "いいえ"}')

# 結果保存
all_store_result = {
    '分析タイプ': '全店舗統合',
    '店舗': '全店舗',
    'データ数': len(model_data_all),
    'ベストモデル': best_model_all.__class__.__name__,
    'R2_Test': overfitting_all['r2_test'],
    'R2_Train': overfitting_all['r2_train'],
    'R2_Gap': overfitting_all['r2_gap'],
    '過学習': overfitting_all['is_overfitting'],
    '深刻度': overfitting_all['overfitting_severity'],
    'MAE_Test': overfitting_all['mae_test'],
    'RMSE_Test': overfitting_all['rmse_test']
}

# Learning Curve
fig_all, gap_all = plot_learning_curve(best_model_all, X_train_all, y_train_all,
                                       model_name='全店舗統合', cv=10)
fig_all.savefig('output/learning_curves/learning_curve_AllStores.png',
               dpi=150, bbox_inches='tight')
plt.close(fig_all)

print('\n✅ 全店舗統合モデル分析完了')


In [ ]:
# ========================================
# 9. 店舗別個別モデル分析
# ========================================

print('\n' + '='*80)
print('🏪 店舗別個別モデル分析')
print('='*80)

store_results = []

# 店舗リスト取得
stores = data['店舗'].unique()
print(f'\n対象店舗数: {len(stores)}店舗')
print(f'店舗リスト: {stores}')

for store in stores:
    print(f'\n--- 店舗: {store} ---')
    
    # 店舗データ抽出
    store_data = data[data['店舗'] == store].copy()
    
    if len(store_data) < 500:
        print(f'⚠️ データ不足 ({len(store_data)}行) - スキップ')
        continue
    
    print(f'データ数: {len(store_data)}行')
    print(f'カテゴリ数: {store_data["category_l"].nunique()}')
    
    # 不要列除外
    exclude_cols = ['店舗', '商品名', '日付', '売上金額',
                   'フェイスくくり大分類', 'フェイスくくり中分類', 'フェイスくくり小分類']
    
    # カテゴリをダミー変数化
    if 'category_l' in store_data.columns:
        category_dummies = pd.get_dummies(store_data['category_l'], prefix='cat')
    else:
        category_dummies = pd.DataFrame()
    
    feature_cols = [c for c in store_data.columns 
                   if c not in exclude_cols + ['売上数量', 'category_l']]
    numeric_cols = store_data[feature_cols].select_dtypes(include=GPU_MODELS + ['et', 'rf', 'gbr', 'dt']).columns.tolist()
    
    model_data_store = pd.concat([
        store_data[numeric_cols + ['売上数量']],
        category_dummies
    ], axis=1).dropna()
    
    if len(model_data_store) < 100:
        print(f'⚠️ 有効データ不足 ({len(model_data_store)}行) - スキップ')
        continue
    
    print(f'有効データ: {len(model_data_store)}行, {len(model_data_store.columns)-1}特徴量')
    
    # PyCaret setup
    s_store = setup(
        model_data_store,
        target='売上数量',
        session_id=123,
        train_size=0.8,
        fold=5,
        normalize=True,
        remove_multicollinearity=True,
        multicollinearity_threshold=0.95,
        silent=True,
        verbose=False
    )
    
    # モデル比較
    print(f'\n🔍 # GPU高速化: XGBoost/CatBoost GPUを優先使用
    # 注: Top 20特徴量が自動的に考慮されます（PyCaret feature_importance）
    compare_models()実行中（店舗: {store}）...')
    best_models_store = # GPU高速化: XGBoost/CatBoost GPUを優先使用
    compare_models(
        n_select=5,
        sort='R2',
        turbo=False,
        verbose=False
    )
    
    comparison_store = pull()
    print('\n📊 モデル比較結果（Top 5）:')
    print(comparison_store[['Model', 'R2', 'MAE', 'RMSE']].head())
    
    # 過学習検出
    best_model_store = best_models_store[0] if isinstance(best_models_store, list) else best_models_store
    
    X_train_store = get_config('X_train')
    y_train_store = get_config('y_train')
    X_test_store = get_config('X_test')
    y_test_store = get_config('y_test')
    
    overfitting_store = detect_overfitting(
        best_model_store, X_train_store, y_train_store, X_test_store, y_test_store,
        model_name=f'{store} - {best_model_store.__class__.__name__}'
    )
    
    print(f'\n🔬 過学習検出結果:')
    print(f'  Train R²: {overfitting_store["r2_train"]:.4f}')
    print(f'  Test R²: {overfitting_store["r2_test"]:.4f}')
    print(f'  R²ギャップ: {overfitting_store["r2_gap"]:.4f}')
    print(f'  過学習判定: {"はい" if overfitting_store["is_overfitting"] else "いいえ"}')
    
    # 結果保存
    store_results.append({
        '分析タイプ': '店舗別',
        '店舗': store,
        'データ数': len(model_data_store),
        'ベストモデル': best_model_store.__class__.__name__,
        'R2_Test': overfitting_store['r2_test'],
        'R2_Train': overfitting_store['r2_train'],
        'R2_Gap': overfitting_store['r2_gap'],
        '過学習': overfitting_store['is_overfitting'],
        '深刻度': overfitting_store['overfitting_severity'],
        'MAE_Test': overfitting_store['mae_test'],
        'RMSE_Test': overfitting_store['rmse_test']
    })
    
    # Learning Curve
    fig_store, gap_store = plot_learning_curve(
        best_model_store, X_train_store, y_train_store,
        model_name=f'店舗{store}', cv=5
    )
    fig_store.savefig(f'output/learning_curves/learning_curve_Store_{store}.png',
                     dpi=150, bbox_inches='tight')
    plt.close(fig_store)
    
    print(f'✅ 店舗{store}分析完了')

print(f'\n✅ 店舗別分析完了: {len(store_results)}店舗')


In [ ]:
# ========================================
# 10. 全店舗統合 vs 店舗別 比較分析
# ========================================

print('\n' + '='*80)
print('📊 全店舗統合 vs 店舗別モデル 比較分析')
print('='*80)

# 全結果を統合
store_comparison_results = [all_store_result] + store_results
store_comparison_df = pd.DataFrame(store_comparison_results)

if len(store_comparison_df) > 0:
    print('\n【全店舗統合 vs 店舗別 パフォーマンス比較】')
    print(store_comparison_df[['分析タイプ', '店舗', 'ベストモデル', 'R2_Test',
                              'R2_Gap', '過学習', '深刻度', 'データ数']].to_string(index=False))
    
    # 統計サマリー
    print('\n' + '='*80)
    print('📈 戦略別パフォーマンスサマリー')
    print('='*80)
    
    strategy_summary = store_comparison_df.groupby('分析タイプ').agg({
        'R2_Test': ['mean', 'std', 'min', 'max'],
        'R2_Gap': ['mean', 'max'],
        '過学習': lambda x: (x == True).sum(),
        '店舗': 'count'
    }).round(4)
    
    print(strategy_summary)
    
    # 最適戦略の判定
    all_store_r2 = store_comparison_df[store_comparison_df['分析タイプ'] == '全店舗統合']['R2_Test'].iloc[0]
    per_store_avg_r2 = store_comparison_df[store_comparison_df['分析タイプ'] == '店舗別']['R2_Test'].mean()
    
    print('\n' + '='*80)
    print('🎯 最適モデリング戦略の判定')
    print('='*80)
    
    print(f'\n全店舗統合モデル R²: {all_store_r2:.4f}')
    print(f'店舗別モデル平均 R²: {per_store_avg_r2:.4f}')
    print(f'差分: {per_store_avg_r2 - all_store_r2:.4f} ({(per_store_avg_r2 - all_store_r2)/all_store_r2:.2%})')
    
    # 判定基準
    if per_store_avg_r2 > all_store_r2 + 0.05:  # 5%以上改善
        recommendation = '✅ 店舗別モデル推奨'
        reason = f'店舗別モデルが{(per_store_avg_r2 - all_store_r2)/all_store_r2:.1%}改善')
    elif per_store_avg_r2 < all_store_r2 - 0.05:  # 5%以上悪化
        recommendation = '✅ 全店舗統合モデル推奨'
        reason = f'統合モデルが{(all_store_r2 - per_store_avg_r2)/all_store_r2:.1%}優位')
    else:  # ±5%以内
        recommendation = '⚖️ ハイブリッド戦略推奨'
        reason = '差分が小さいため、店舗特性に応じて使い分け'
    
    print(f'\n{recommendation}')
    print(f'理由: {reason}')
    
    # 詳細推奨
    print('\n💡 実装推奨:')
    if '店舗別' in recommendation:
        print('  1. 各店舗で個別モデルを学習・デプロイ')
        print('  2. 新規店舗は統合モデルで開始、データ蓄積後に個別化')
        print('  3. 定期的にモデル再学習（月次推奨）')
    elif '統合' in recommendation:
        print('  1. 全店舗で1つのモデルを共有（メンテナンスコスト削減）')
        print('  2. 店舗ダミー変数で店舗特性を吸収')
        print('  3. 店舗固有の特徴量を追加検討（立地、商圏など）')
    else:  # ハイブリッド
        print('  1. データ量が豊富な店舗 → 個別モデル')
        print('  2. データ不足の店舗 → 統合モデル')
        print('  3. 閾値: 1店舗あたり5000行以上なら個別化')
    
    # 店舗別パフォーマンスランキング
    print('\n' + '='*80)
    print('🏆 店舗別パフォーマンスランキング')
    print('='*80)
    
    store_only = store_comparison_df[store_comparison_df['分析タイプ'] == '店舗別'].copy()
    if len(store_only) > 0:
        store_only_sorted = store_only.sort_values('R2_Test', ascending=False)
        print(store_only_sorted[['店舗', 'ベストモデル', 'R2_Test', 'R2_Gap',
                                '過学習', 'データ数']].to_string(index=False))
        
        # ベスト店舗とワースト店舗
        best_store = store_only_sorted.iloc[0]
        worst_store = store_only_sorted.iloc[-1]
        
        print(f'\n🥇 最高精度店舗: {best_store["店舗"]} (R²={best_store["R2_Test"]:.4f})')
        print(f'   モデル: {best_store["ベストモデル"]}')
        print(f'   データ数: {best_store["データ数"]:,}行')
        
        print(f'\n⚠️ 改善必要店舗: {worst_store["店舗"]} (R²={worst_store["R2_Test"]:.4f})')
        print(f'   モデル: {worst_store["ベストモデル"]}')
        print(f'   データ数: {worst_store["データ数"]:,}行')
        print(f'   改善策: データ期間延長、特徴量見直し、統合モデル利用検討')
    
    # CSV保存
    output_path = Path('output/store_comparison_results.csv')
    store_comparison_df.to_csv(output_path, index=False, encoding='utf-8-sig')
    print(f'\n✅ 結果をCSV保存: {output_path}')

else:
    print('⚠️ 比較結果が得られませんでした')

print('\n✅ 店舗別分析完了！')
